I am going to attempt to develop a model that will predict who will win any individaul mens ncaa college basketball game. Then, I would like to see how accuratly the model can predict Michigan States record from the 2019 season. I chose 2019 because MSU made the final four so it seemed like the most exciting option.

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

I got the data from https://barttorvik.com/trank.php?year=2021&sort=&top=0&conlimit=All#

In [14]:
data = pd.read_csv("cbb19.csv")

In [15]:
data

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED
0,Gonzaga,WCC,37,33,123.4,89.9,0.9744,59.0,44.2,14.9,...,35.3,25.9,61.4,43.4,36.3,30.4,72.0,7.0,E8,1.0
1,Virginia,ACC,38,35,123.0,89.9,0.9736,55.2,44.7,14.7,...,29.1,26.3,52.5,45.7,39.5,28.9,60.7,11.1,Champions,1.0
2,Duke,ACC,38,32,118.9,89.2,0.9646,53.6,45.0,17.5,...,33.2,24.0,58.0,45.0,30.8,29.9,73.6,11.2,E8,1.0
3,North Carolina,ACC,36,29,120.1,91.4,0.9582,52.9,48.9,17.2,...,30.2,28.4,52.1,47.9,36.2,33.5,76.0,10.0,S16,1.0
4,Michigan,B10,37,30,114.6,85.6,0.9665,51.6,44.1,13.9,...,27.5,24.1,51.8,44.3,34.2,29.1,65.9,9.2,S16,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Alcorn St.,SWAC,27,10,89.0,112.6,0.0628,45.7,52.7,24.1,...,30.5,36.5,45.0,55.3,31.3,32.1,67.1,-16.7,NaN,NaN
349,New Hampshire,AE,27,5,83.7,106.1,0.0613,44.0,51.5,18.4,...,21.9,38.0,39.4,52.1,32.6,33.6,67.1,-20.2,NaN,NaN
350,Chicago St.,WAC,30,3,88.5,117.3,0.0380,44.2,57.8,22.5,...,33.1,33.9,43.5,57.9,30.7,38.5,71.9,-20.9,NaN,NaN
351,Delaware St.,MEAC,29,6,84.3,112.2,0.0358,40.0,52.4,19.0,...,25.5,39.2,37.7,52.6,29.0,34.7,71.6,-21.7,NaN,NaN


Data Set Variable Abreviations:


ADJOE: Adjusted Offensive Efficiency (Points per 100 possessions)


ADJDE: Ajusted Defensive Efficiency (Points allowed per 100 possesions)


TOR: Offensive Turn Over Rate (Per 100 possesions)


TORD: Defensive Turnover Rate (Steals)


EFG_O: Effective Field Goal Percentage Shot


EFG_D: Effective Field Goal Percentage Allowed


FTR: Free Throws Percentage Made


FTRD: Free Throw Percentage Allowed


ORB: Offensive Rebound Percentage


DRB Defensive Rebound Percentage


2P_O: Two Point Shot Percentage Made


2P_D: Two Point Shot Percentage Allowed


3P_O: Three Point Shot Percentage Made


3P_D: Three Point Shot Percentage Allowed


ADJ_T: Adjusted Tempo (Possesions Per 40 Minutes)

WAB: Wins Above Bubble

In [19]:
data = data.drop_duplicates(subset = "TEAM", keep="first")

I wanted to remove NA values, but realized if I were to do that, it would remove all of the teams who did not make the NCAA tournament at the end of the year. Instead, I just filled those values with a string for "missed tournament"

In [22]:
data.fillna({'SEED':'Missed Tournament'}, inplace = True)
data.SEED.value_counts(dropna=False)
data.fillna({'POSTSEASON':'Missed Tournament'}, inplace = True)
data.SEED.value_counts(dropna=False)

Missed Tournament    285
16.0                   6
11.0                   6
10.0                   4
15.0                   4
14.0                   4
13.0                   4
12.0                   4
1.0                    4
2.0                    4
8.0                    4
7.0                    4
6.0                    4
5.0                    4
4.0                    4
3.0                    4
9.0                    4
Name: SEED, dtype: int64

In [23]:
data

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED
0,Gonzaga,WCC,37,33,123.4,89.9,0.9744,59.0,44.2,14.9,...,35.3,25.9,61.4,43.4,36.3,30.4,72.0,7.0,E8,1.0
1,Virginia,ACC,38,35,123.0,89.9,0.9736,55.2,44.7,14.7,...,29.1,26.3,52.5,45.7,39.5,28.9,60.7,11.1,Champions,1.0
2,Duke,ACC,38,32,118.9,89.2,0.9646,53.6,45.0,17.5,...,33.2,24.0,58.0,45.0,30.8,29.9,73.6,11.2,E8,1.0
3,North Carolina,ACC,36,29,120.1,91.4,0.9582,52.9,48.9,17.2,...,30.2,28.4,52.1,47.9,36.2,33.5,76.0,10.0,S16,1.0
4,Michigan,B10,37,30,114.6,85.6,0.9665,51.6,44.1,13.9,...,27.5,24.1,51.8,44.3,34.2,29.1,65.9,9.2,S16,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Alcorn St.,SWAC,27,10,89.0,112.6,0.0628,45.7,52.7,24.1,...,30.5,36.5,45.0,55.3,31.3,32.1,67.1,-16.7,Missed Tournament,Missed Tournament
349,New Hampshire,AE,27,5,83.7,106.1,0.0613,44.0,51.5,18.4,...,21.9,38.0,39.4,52.1,32.6,33.6,67.1,-20.2,Missed Tournament,Missed Tournament
350,Chicago St.,WAC,30,3,88.5,117.3,0.0380,44.2,57.8,22.5,...,33.1,33.9,43.5,57.9,30.7,38.5,71.9,-20.9,Missed Tournament,Missed Tournament
351,Delaware St.,MEAC,29,6,84.3,112.2,0.0358,40.0,52.4,19.0,...,25.5,39.2,37.7,52.6,29.0,34.7,71.6,-21.7,Missed Tournament,Missed Tournament


In [24]:
msu_2019_schedule = ['Kansas', 'Florida Gulf Coast', 'Louisiana Monroe', 'Tennessee Tech', 'UCLA', 'Texas', 'Louisville', 'Rutgers', 'Iowa', 'Florida', 'Green Bay', 'Oakland', 'Northern Illinois', 'Northwestern', 'Ohio State', 'Purdue', 'Penn State', 'Nebraska', "Maryland", 'Iowa', 'Purdue', 'Indiana', 'Illinois', 'Minnesota', 'Wisconsin', 'Ohio State', 'Rugtgers', 'Michigan', 'Indiana', 'Nebraska', 'Michigan']



win percentage, rebounds, assists, turnovers, steals, blocks, offensive rating, defensive rating, and true shooting percentage.

In [34]:
data2 = data.drop('W', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(data2,data['W'], test_size=0.25, random_state=400)

In [35]:
svc = SVC(C=1.0, random_state=1, kernel='linear')
svc.fit(x_train, y_train)

ValueError: could not convert string to float: 'UNC Greensboro'